<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>Автор материала: Егор Лабинцев – @egor_labintcev

<img src="https://habrastorage.org/web/04d/883/420/04d8834204974f0baf14dc277b634e16.jpg"/>

                         **Случайная картинка из выдачи google по запросу "небаланс классов" :**

## Краткая постановка проблемы

В задачах классификации баланс классов часто нарушается, и обычно именно меньший класс является целевым.

Фрод, отказы техники, положительные медицинские диагнозы, нежелательная выдача в поисковике, отток -- лишь часть примеров таких задач. 

Почему обычные алгоритмы (без шаманства) не слишком хорошо работают?

Если в общих чертах, то дело в том, что внутри многих алгоритмов зашита какая-либо оптимизация [loss](https://en.wikipedia.org/wiki/Loss_function)-функции, часто не учитывающей баланс классов в выборке. Именно поэтому модель стремится предсказать как можно **больше** объектов бОльшего класса, игнорируя меньший класс, но снижая общий error-rate.

В этом tutorial мы рассмотрим некоторые методы, которые позволяют бороться с проблемой неравных классов. 
План такой:

* Внутренние ручки алгоритмов (+ алгоритм для несбалансированных классов)
* Библиотека imbalanced-learn

В качестве модельного датасета будут выступать данные о раздачах в покере (hand), где признаками будут являться карты (масть -- Suit, ранг -- C), а target -- Poker Hand, т.е. различные комбинации имеющихся карт. Датасет можно скачать [здесь](https://archive.ics.uci.edu/ml/datasets/Poker+Hand)

Выдержка из описания features датасета:

```
1) S1 "Suit of card #1" 
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs} 

2) C1 "Rank of card #1" 
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King
```

Выдержка о target из описания [датасета](https://archive.ics.uci.edu/ml/datasets/Poker+Hand):

```
0: Nothing in hand; not a recognized poker hand
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush 
```

Я не большой спец в покере, но чем больше цифра, тем реже класс и тем выигрышнее позиция.

Для примеров нам понадобится .py [скрипт](https://github.com/silicon-valley-data-science/learning-from-imbalanced-classes/blob/master/blagging.py) blagging.py, который можно просто положить рядом с ноутбуком, а также библиотека [imbalanced-learn](http://contrib.scikit-learn.org/imbalanced-learn/index.html):

`pip install -U imbalanced-learn`

или для Anaconda 

`conda install -c glemaitre imbalanced-learn`

In [ ]:
# Загрузка библиотек

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import neighbors
from sklearn.ensemble import (ExtraTreesClassifier, GradientBoostingClassifier,
                              RandomForestClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (auc, precision_recall_curve, roc_auc_score,
                             roc_curve)
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 8)

## Внутренние ручки алгоритмов

Загрузим данные и посмотрим на них:

In [ ]:
def load_and_prepare_data():
    
    # Загрузим данные

    df = pd.read_csv('poker-hand-training-true.data', 
                     names=['Suit1', 'C1', 'Suit2', 'C2', 'Suit3', 
                            'C3', 'Suit4', 'C4', 'Suit5', 'C5', 'CLASS'])
    
    # кодирование порядковых (ordinal) признаков -- отдельная тема, здесь обойдемся one-hot

    ordinal_columns = [col for col in df.columns if 'Suit' in col]

    ohe = OneHotEncoder(sparse=False)
    encoded_ordinal = ohe.fit_transform(df[ordinal_columns])

    # удаляем оригинальные колонки
    df.drop(ordinal_columns, axis=1, inplace=True)
    
    tmp = pd.DataFrame(encoded_ordinal, columns=['S ' + str(i) for i in range(encoded_ordinal.shape[1])])
    df = pd.concat([df, tmp], axis=1)
    
    return df

In [ ]:
df = load_and_prepare_data()

In [ ]:
df.head(10)

Взглянем на распределение классов в выборке, чтобы оценить несбалансированность данных.

In [ ]:
# Распределение классов в выборке

print("Initial class percentages: \n")
df.CLASS.value_counts()

In [ ]:
X = df.drop('CLASS', axis=1).as_matrix()
y = df.CLASS

Посмотрим на качество алгоритмов as is, предварительно разбив данный на train и test.

В процессе просмотра метрик рекомендую особенно обращать внимание на `recall` классов или на F1-меру.

In [ ]:
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, 
                                                    test_size=0.4, random_state=RANDOM_STATE)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
clf = RandomForestClassifier(random_state=RANDOM_STATE)

clf.fit(X_train, y_train)

print(classification_report(y_test, clf.predict(X_test)))

Как мы видим, наиболее редкие классы дефолтный RandomForest не нашел и ругается на отсутствие предиктов по ним.

Представим на секунду, что нас интересуют классы с 6 по 9 включительно, т.е. редкие, но резко повышающие вероятность победить. Посмотрим, возможно ли подобрать веса таким образом, чтобы найти эти классы.

В некоторых алгоритмах существует возможность проставить `class_weight`, например, у всеми любимой логистической регрессии и случайного леса и таким образом скорректировать штраф за неверно предсказанный объект. Альтернативой ручном подбору является `'balanced'` опция, проставляющая веса в соотвествии с распределением в обучающей выборке.

In [ ]:
searching_for_classes = ['balanced', 
                         {6:2, 7:2, 8:2, 9:2},
                         {6:10, 7:10, 8:10, 9:10}
                        ]

In [ ]:
for option in searching_for_classes:
    
    clf = RandomForestClassifier(class_weight=option, random_state=RANDOM_STATE)
    clf.fit(X_train, y_train)

    print(classification_report(y_test, clf.predict(X_test)))

Теперь посмотрим на ExtraTreesClassifier

In [ ]:
for option in searching_for_classes:
    
    clf = ExtraTreesClassifier(class_weight=option, random_state=RANDOM_STATE)
    clf.fit(X_train, y_train)

    print('weights: ' + str(option) + '\n' + classification_report(y_test, clf.predict(X_test)) + '\n' )

Видим, что в первом случае (при "'balanced'") мы теперь находим 5-ый класс. Впрочем, это не совсем то, чего мы хотели. Посмотрим на вероятности, проставленные классификатором для каждого из классов.

In [ ]:
predicted_probs = clf.predict_proba(X_test)

In [ ]:
pd.DataFrame(predicted_probs, 
             columns=['prob_' + str(i) for i in range(0,10)]) \
             [["prob_6", "prob_7", "prob_8", "prob_9"]].describe()

Можно было бы воспользоваться стандартным приемом и выкрутить порог по вероятностям, т.е. назначать класс при меньшем, чем дефолтный 0.5 пороге, поймав часть экземпляров класса. Однако мы видим, что в данном случае такой подход не имеет смысла -- вероятности крайне малы и таких примеров совсем немного.

Мы еще вернемся к проблеме определения столь малых классов.

### Blagging Classifier

Теперь посмотрим на работу Blagging Classifier'а, который из коробки умеет балансировать классы.

Отличное интуитивное представление о работе этого классификатора даст [этот](https://github.com/silicon-valley-data-science/learning-from-imbalanced-classes/blob/master/Gaussians.ipynb) ноутбук, а саму статью с подходом можно найти [здесь](https://pdfs.semanticscholar.org/a8ef/5a810099178b70d1490a4e6fc4426b642cde.pdf).

Ну и исходный [код](https://github.com/silicon-valley-data-science/learning-from-imbalanced-classes/blob/master/blagging.py), конечно.


В общий чертах подход следующий:

* Bootstrap из датасета
* Балансирование путем уменьшения размера большего класса (downsampling)
* Обучение Decision Tree на каждой из выборок
* Majority vote по набору деревьев

<img src="https://habrastorage.org/web/29a/31c/af6/29a31caf67f8449dace109394b8b7e6a.png"/>

Картинка [отсюда](https://svds.com/learning-imbalanced-classes/)

Жаль, но этот классификатор работает только для бинарной классификации, так что мы сведем задачу к такому виду. Пусть у нас есть редкие, но выигрышные классы и несколько классов с наибольшим количеством примеров, не являющиеся выигрышными.

In [ ]:
df = load_and_prepare_data()

In [ ]:
# еще раз посмотрим на распределение классов

df.CLASS.value_counts()

In [ ]:
def make_binary(original_data, pos_classes):
    return np.array([(1 if val in pos_classes else 0)
                     for val in original_data ])

In [ ]:
binary_y = make_binary(df.CLASS, set((4, 5, 6, 7, 8, 9)))
print("After merging classes {0, 1, 2, 3} -> 0 and {4, 5, 6, 7, 8, 9} -> 1 \n")

np.unique(binary_y, return_counts=True)

Проблема осталось крайне несбалансированной, давайте проверим как поведут себя алгоритмы sklearn и сравним с BlaggingClassifier'ом.

In [ ]:
X = df.drop('CLASS', axis=1).as_matrix()
y = binary_y

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, 
                                                    test_size=0.4, random_state=RANDOM_STATE)

RandomForest:

In [ ]:
clf = RandomForestClassifier(random_state=RANDOM_STATE)

clf.fit(X_train, y_train)

print(classification_report(y_test, clf.predict(X_test)))

GradientBoosting:

In [ ]:
clf = GradientBoostingClassifier(random_state=RANDOM_STATE)

clf.fit(X_train, y_train)

print(classification_report(y_test, clf.predict(X_test)))

По-прежнему никаких значительных улучшений, даже после merge классов.

In [ ]:
from blagging import BlaggingClassifier

clf = BlaggingClassifier(random_state=RANDOM_STATE)

clf.fit(X_train, y_train)

print(classification_report(y_test, clf.predict(X_test)))

Как мы видим, `BlaggingClassifier` отлично показал себя, выдав приличную полноту для такой задачи.

Для меня был сюрпризом результат ExtraTreesClassifier:

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier(random_state=RANDOM_STATE)

clf.fit(X_train, y_train)

print(classification_report(y_test, clf.predict(X_test)))

Можно подумать (но мы не будем) о том, как объединить предсказания ExtraTreesClassifier и BlaggingClassifier для лучшего результата.

Давайте теперь посмотрим на AUC-ROC для каждого из классификаторов.
Надо заметить, что стоит строить AUC-ROC на кросс-валидации по фолдам, т.к. некоторые из объектов могут быть нетипичными для класса и это будет заметно при разбиении, а также оценка будет менее смещенной, но мы ограничимся разделением на train-test для демонстрации методов.

In [ ]:
clfs = [
        ['RandomForestClassifier', RandomForestClassifier(random_state=RANDOM_STATE)],
        ['GradientBoostingClassifier', GradientBoostingClassifier(random_state=RANDOM_STATE)],
        ['ExtraTreesClassifier', ExtraTreesClassifier(random_state=RANDOM_STATE)], 
        ['BlaggingClassifier', BlaggingClassifier(random_state=RANDOM_STATE)]
       ]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

for name, clf in clfs:
    
    clf.fit(X_train, y_train)
    fpr, tpr, thresholds = roc_curve(y_test, clf.predict_proba(X_test)[:, 1])
    roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
    plt.plot(fpr, tpr, linestyle='-',
             label='{} (area = %0.2f)'.format(name) % roc_auc)
    
plt.plot([0, 1], [0, 1], linestyle='--', color='k',
         label='Random')    
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()
ax.spines['left'].set_position(('outward', 10))
ax.spines['bottom'].set_position(('outward', 10))
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')

plt.legend(loc="lower right")

plt.show()    

Видно, что за счет "сваливания предсказаний классификатора в больший класс, градиентный бустинг выигрывает у BlaggingClassifier'а.

Но давайте посмотрим на AUC-PR и полноту по редкому классу и всё встанет на свои места.

In [ ]:
from sklearn.metrics import recall_score

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

for name, clf in clfs:
    
    clf.fit(X_train, y_train)
    fpr, tpr, thresholds = precision_recall_curve(y_test, clf.predict_proba(X_test)[:, 1])
    recall_1 = recall_score(y_test, clf.predict(X_test))
    plt.plot(fpr, tpr, linestyle='-',
             label='{} (recall_1 = %0.2f)'.format(name) % recall_1)
    
    
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()
ax.spines['left'].set_position(('outward', 10))
ax.spines['bottom'].set_position(('outward', 10))
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')

plt.legend(loc="upper right")

plt.show()    

Здесь мы видим, что наибольшая `recall` меньшего класса именно у Blagging Classifier'а.

## Библиотека imbalanced-learn

[Библиотека](http://contrib.scikit-learn.org/imbalanced-learn/install.html) imbalanced-learn позволяет использовать различные техники сэмплирования (как over, так и under, а также их комбинации). Позади некоторых техник стоит нетривиальные подходы, не влезающие в данный туториал, но я оставлю ссылки.

В библиотеку входят:

* Under-sampling methods. Всё просто, сэмплируем из б**о**льшего класса для выравнивания выборки по меньшему классу. Возможны два варианта: 
 - генерация новых примеров из большего класса на основе [центроид](http://contrib.scikit-learn.org/imbalanced-  learn/generated/imblearn.under_sampling.ClusterCentroids.html) кластеров;
 - [выбор](http://contrib.scikit-learn.org/imbalanced-learn/api.html#module-imblearn.under_sampling.prototype_selection) примеров из большего класса разными способами (их реально много)


* Over-sampling methods. Тут тоже всё просто -- мы добавляем в датасет примеры меньшего класса, просто [копируя](http://contrib.scikit-learn.org/imbalanced-learn/generated/imblearn.over_sampling.RandomOverSampler.html) или используя более хитрые техники как, например, [SMOTE](http://contrib.scikit-learn.org/imbalanced-learn/generated/imblearn.over_sampling.SMOTE.html), который позволяет генерировать синтетические примеры на основе близости нескольких соседей в признаковом пространстве, создавая (с включением случайности) новый вектор признаков для нового примера. [Тут](https://www.cs.cmu.edu/afs/cs/project/jair/pub/volume16/chawla02a-html/node6.html#SECTION00042000000000000000) подробнее.

Давайте посмотрим как поведут себя эти методы на оригинальном датасете при наличии всех классов.

In [ ]:
df = load_and_prepare_data()

X = df.drop('CLASS', axis=1).as_matrix()
y = df.CLASS

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, 
                                                    test_size=0.4, random_state=RANDOM_STATE)

In [ ]:
df.CLASS.value_counts()

Начнем с простого -- обычный undersampling:

In [ ]:
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import RandomUnderSampler

pipe = make_pipeline(RandomUnderSampler(random_state=RANDOM_STATE), 
                     ExtraTreesClassifier(random_state=RANDOM_STATE))

In [ ]:
pipe.fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

Уже неплохо (5,6 и 7 классы), но мы сильно просели по большим классам.

Попробуем [CondensedNearestNeighbour](http://machinelearning.org/proceedings/icml2005/papers/004_Fast_Angiulli.pdf):

In [ ]:
from imblearn.under_sampling import CondensedNearestNeighbour

pipe = make_pipeline(CondensedNearestNeighbour(random_state=RANDOM_STATE), 
                     ExtraTreesClassifier(random_state=RANDOM_STATE))

pipe.fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

Уже лучше, мы снова видим большие классы!

Надо заметить, что многие из методов, доступных в imbalanced-learn не работает для мультиклассовой постановки задачи. Поэтому вернемся к бинарной постановке для демонстрации подхода over-sampling.

In [ ]:
df = load_and_prepare_data()
binary_y = make_binary(df.CLASS, set((4, 5, 6, 7, 8, 9)))

print("After merging classes {0, 1, 2, 3} -> 0 and {4, 5, 6, 7, 8, 9} -> 1 \n")

np.unique(binary_y, return_counts=True)
X = df.drop('CLASS', axis=1).as_matrix()
y = binary_y

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, 
                                                    test_size=0.4, random_state=RANDOM_STATE)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

pipe = make_pipeline(RandomOverSampler(random_state=RANDOM_STATE), 
                     ExtraTreesClassifier(random_state=RANDOM_STATE))

pipe.fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

Даже обычный over-sampling справился неплохо.

In [ ]:
from imblearn.over_sampling import SMOTE

pipe = make_pipeline(SMOTE(random_state=RANDOM_STATE), 
                     ExtraTreesClassifier(random_state=RANDOM_STATE))

pipe.fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

Надо заметить, что imblearn имеет свою функции оценки качества модели, включающую precision, recall, specificity (true negative rate), f1, геометрическое среднее recall (sensitivity) и specificity, а также index balanced [accuracy](http://repositori.uji.es/xmlui/bitstream/handle/10234/23961/33068.pdf?sequence=1).

Последний рассчитывается следующим образом:

$$ IBA = (1 + Dominance)· Gmean^2 ,$$
    где
$$ Dominance = True Positive Rate - True Negative Rate $$

In [ ]:
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, pipe.predict(X_test)))

Напоследок построим ROC-кривые для некоторых методов.

In [ ]:
class DummySampler(object):

    def sample(self, X, y):
        return X, y

    def fit(self, X, y):
        return self

    def fit_sample(self, X, y):
        return self.sample(X, y)

In [ ]:
classifier = ['ExtraTreesClassifier', ExtraTreesClassifier()]

samplers = [
    ['Standard', DummySampler()],
    ['SMOTE', SMOTE(random_state=RANDOM_STATE)],
    ['RandomOverSampler', RandomOverSampler(random_state=RANDOM_STATE)],
    ['RandomUnderSampler', RandomUnderSampler(random_state=RANDOM_STATE)]
]

pipelines = [
    ['{}-{}'.format(sampler[0], classifier[0]),
     make_pipeline(sampler[1], classifier[1])]
    for sampler in samplers
]


fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

for name, clf in pipelines:
    
    clf.fit(X_train, y_train)
    fpr, tpr, thresholds = roc_curve(y_test, clf.predict_proba(X_test)[:, 1])
    roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
    plt.plot(fpr, tpr, linestyle='-',
             label='{} (area = %0.2f)'.format(name) % roc_auc)
    
plt.plot([0, 1], [0, 1], linestyle='--', color='k',
         label='Random')    
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()
ax.spines['left'].set_position(('outward', 10))
ax.spines['bottom'].set_position(('outward', 10))
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')

plt.legend(loc="lower right")

plt.show()    

## Выводы

* Практика -- залог успеха. Пробуйте различные подходы и алгоритмы для решения вашей задачи, комбинируйте их
* Перепроверяйте метрики, смотрите на задачу под правильным углом -- вы не тюните метрику, а решаете задачу

Ссылки:

* [Документация](http://contrib.scikit-learn.org/imbalanced-learn/index.html) imbalanced-learn
* [Пост](https://svds.com/learning-imbalanced-classes/) про работу с несбалансированными выборками, [FAQ](https://svds.com/imbalanced-classes-faq/) по ним и их [репозиторий](https://github.com/silicon-valley-data-science), откуда я взял Blagging Classifier и часть кода

